<a href="https://colab.research.google.com/github/Svanzi/Jazz-Improvisation/blob/main/Jazz_Improv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import sys
import re
import numpy as np
import pandas as pd
import music21
from glob import glob
import IPython
from tqdm import tqdm
import pickle
from tensorflow.keras.utils import to_categorical
# import play
import os

In [65]:
!git clone https://github.com/Svanzi/Jazz-Improvisation.git
%cd Jazz-Improvisation

Cloning into 'Jazz-Improvisation'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 46 (delta 12), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (46/46), 39.16 KiB | 2.30 MiB/s, done.
Resolving deltas: 100% (12/12), done.
/content/Jazz-Improvisation/Jazz-Improvisation/Jazz-Improvisation/Jazz-Improvisation/Jazz-Improvisation


In [43]:
from music21 import converter, instrument, note, chord, stream

In [66]:
songs = glob("MIDI/*.mid")
# Per verificare che il caricamento dei file sia andato a buon fine e che i file midi siano presenti
print(songs)

['MIDI/Nintendo_-_Dr._Mario.mid', 'MIDI/Cameron_Lee_Simpson_-_Illumination_Nights.mid', 'MIDI/Jacob_Allen_-_Test.mid', 'MIDI/Andre_and_Schwandt_-_Dream_a_Little_Dream.mid', 'MIDI/michel_petrucciani_caravan_solo_-_michel_petrucciani_caravan_solo.mid', 'MIDI/Moab_Berckmans_de_Oliveira_-_MBOM06.mid', 'MIDI/Ensemble-_Isaac_Piano_Week_4_with_structure__-_Ensamble__Geo_Drum_110_Bpm.mid']


## Load all the notes in an array

In [67]:
def get_notes():
  notes = []
  for file in songs:
    midi = converter.parse(file)
    notes_to_parse = []

    try:
      parts = instrument.partitionByInstrument(midi)
    except:
      pass
    if parts: # file has instrument parts
      notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
      notes_to_parse = midi.flat.notes

    for element in  notes_to_parse:
      if isinstance(element, note.Note):
        notes.append(str(element.pitch))
      elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))

  return notes

## Define the input and output sequences

In [75]:
# n_vocab represent the total number of unique notes in the variable 'notes_data'
def sequences(notes, n_vocab):
  sequence_length = 100

  pitchnames = sorted(set(item for item in notes))

  note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

  network_input = []
  network_output = []

  for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i: i + sequence_length]
    sequence_out = note[i + sequence_length]
    network_input.append(note_to_int[char] for char in sequence_in)
    network_output.append(note_to_int[sequence_out])

  n_patterns = len(network_input)

  network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

  network_input = network_input / float(n_vocab)

  network_output = np_utils.to_categorical(network_output)

  return (network_input, network_output)

## Create the Network

In [ ]:
from keras.model import Sequential
from keras.model import Activation, Dense, Dropout, LSTM, Flatten

def create_network(network_input, n_vocab):
  """Create the model architecture"""
  model = Sequential()
  model.add(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
  model.add(Dropout(0.3))
  model.add(LSTM(512, return_sequence=True))
  model.add(Dropout(0.3))
  model.add(LSTM(256))
  model.add(Dense(256))
  model.add(Dropout(0.3))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

  return model

## Train

In [ ]:
from keras.callbacks import ModelCheckpoint

def train(model, network_input, network_output, epochs):
  """Train the neural network"""

  filepath = "weights-improvement-{epoch:02d}-{loss:4.f}-bigger.hdf5"

  checkpoint = ModelCheckpoint(
      filepath,
      monitor = 'loss',
      verbose = 0,
      save_best_only = True,
      mode = 'min'
  )

  callbacks_list = [checkpoint]

  model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks = callbacks_list)

In [73]:
def train_network():
  """
  Get notes
  Generates input and putput sequences
  Create model
  TRains the model for the given epochs
  """

  epochs = 200

  notes_data = get_notes()
  print('Notes processed')

  n_vocab = len(set(notes_data))
  print('Vocab generated')

  network_in, network_out = sequences(notes_data, n_vocab)
  print('Input and Output processed')

  model = create_network(network_in, n_vocab)

In [74]:
print(f"Extracted {len(notes_data)} notes/chords.")
print("First 20 entries:", notes_data[:20])

Extracted 3133 notes/chords.
First 10 entries: ['7.9.0.3', 'C4', '3.7', '0.2', 'E-4', 'C4', '7.9.0.3', '3.7', '0.2', '5.8.10.1', 'C4', '7.9.0.3', '3.7', '0.2', 'E-4', 'C4', '7.9.0.3', '3.7', '0.2', '6.8.10.2']
